##Data Reading and Preparation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
data = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv"


In [3]:
!wget $data -O data-homework3.csv

--2023-10-01 11:54:13--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data-homework3.csv’

data-homework3.csv  100%[===================>]   1.41M  --.-KB/s    in 0.02s   

2023-10-01 11:54:13 (57.4 MB/s) - ‘data-homework3.csv’ saved [1475504/1475504]



In [3]:
df = pd.read_csv('data-homework3.csv')

In [38]:
X = df[["Make", "Model", "Year","Engine HP", "Engine Cylinders","Transmission Type","Vehicle Style","highway MPG","city mpg"]]

In [39]:
y=df[["MSRP"]]

In [40]:
X.columns = X.columns.str.replace(' ', '_').str.lower()


In [44]:
X.isna().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
dtype: int64

In [42]:
X.fillna(0, inplace=True)

<ipython-input-42-67b4599ed24c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


In [45]:
y.rename(columns = {'MSRP':'price'}, inplace = True)


<ipython-input-45-f53a3ffce32c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.rename(columns = {'MSRP':'price'}, inplace = True)


##Q1

In [46]:
X.transmission_type.value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

##Q.2

In [47]:
X.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
dtype: object

In [15]:
numerical =["year","engine_hp","engine_cylinders", "highway_mpg", "city_mpg"]

In [49]:
matrix =X.corr()

<ipython-input-49-26b75b1faa0c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  matrix =X.corr()


In [50]:
matrix

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


##Q.3

In [51]:
y['above_average']=(y.price>y.price.mean()).astype(int)
y

<ipython-input-51-e9fe0fe52852>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['above_average']=(y.price>y.price.mean()).astype(int)


,price,above_average
0,46135,1
1,40650,1
2,36350,0
3,29450,0
4,34500,0
...,...,...
11909,46120,1
11910,56670,1
11911,50620,1
11912,50920,1


In [52]:
y=y.drop(['price'], axis=1)

In [53]:
from sklearn.model_selection import train_test_split

In [97]:
X_full_train,X_test, y_full_train, y_test = train_test_split(X, y,test_size=0.2, random_state=1)

In [55]:
X_full_train.shape

(9531, 9)

In [56]:
y_full_train.shape

(9531, 1)

In [98]:
X_train,X_val, y_train, y_val =  train_test_split(X_full_train,y_full_train, test_size=0.25,random_state=1)

In [58]:
from sklearn.metrics import mutual_info_score

In [16]:
categorical=["make","model","transmission_type","vehicle_style"]

In [60]:
def mutual_info_churn_score(series):
  return mutual_info_score(y_train.above_average, series)

In [61]:
mi = X_train[categorical].apply(mutual_info_churn_score)
mi = round(mi,2)
mi.sort_values(ascending=False)

model                0.47
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

##Q.4

In [62]:
from sklearn.feature_extraction import DictVectorizer

In [63]:
dv = DictVectorizer(sparse=False)

In [64]:
train_dicts = X_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [ ]:
dv.get_feature_names_out()

In [66]:
val_dicts = X_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [67]:
from sklearn.linear_model import LogisticRegression

In [68]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)


In [69]:
model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [71]:
y_pred

array([0.72017607, 0.00236037, 0.23448726, ..., 0.00248555, 0.9999968 ,
       0.04562852])

In [74]:
len(churn_decision)

2383

In [ ]:
y_val

In [75]:
y_pred= model.predict_proba(X_val)[:,1]
churn_decision = (y_pred >0.5)
(y_val.values==churn_decision).mean()

0.6025082549863181

##Q.5

In [106]:
for f in range(len(feature_names)):
  feature_names = categorical+numerical
  feature_names.pop(f)
  dv = DictVectorizer(sparse=False)
  train_dicts = X_train[feature_names].to_dict(orient='records')
  X_train_f = dv.fit_transform(train_dicts)
  val_dicts = X_val[feature_names].to_dict(orient='records')
  X_val_f = dv.transform(val_dicts)
  model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
  model.fit(X_train_f,y_train.values.ravel())
  y_pred= model.predict_proba(X_val_f)[:,1]
  churn_decision = (y_pred >0.5)
  acc= (y_val.values==churn_decision).mean()
  diff =0.6025082549863181-acc
  print("difference is is %.7f excluding the feature %s" %(diff,feature_names[f]) )


0
difference is is -0.0049219 excluding the feature model
1
difference is is -0.0030289 excluding the feature transmission_type
2
difference is is -0.0005679 excluding the feature vehicle_style
3
difference is is -0.0018930 excluding the feature year
4
difference is is -0.0001893 excluding the feature engine_hp
5
difference is is -0.0024610 excluding the feature engine_cylinders
6
difference is is 0.0000000 excluding the feature highway_mpg
7
difference is is -0.0017037 excluding the feature city_mpg


##Q.6

In [4]:
X = df[["Make", "Model", "Year","Engine HP", "Engine Cylinders","Transmission Type","Vehicle Style","highway MPG","city mpg"]]


In [5]:
X.columns = X.columns.str.replace(' ', '_').str.lower()


In [6]:
X.fillna(0, inplace=True)

<ipython-input-6-67b4599ed24c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


In [7]:
y = df[["MSRP"]]
y.rename(columns = {'MSRP':'price'}, inplace = True)
y = np.log1p(y.values)


<ipython-input-7-22f79618d82c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.rename(columns = {'MSRP':'price'}, inplace = True)


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_full_train,X_test, y_full_train, y_test = train_test_split(X, y,test_size=0.2, random_state=1)

In [10]:
X_train,X_val, y_train, y_val =  train_test_split(X_full_train,y_full_train, test_size=0.25,random_state=1)

In [11]:
X_train

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
5727,Toyota,Highlander Hybrid,2016,280.0,6.0,AUTOMATIC,4dr SUV,28,27
5814,Chevrolet,Impala Limited,2014,300.0,6.0,AUTOMATIC,Sedan,30,18
8450,Buick,Regal,2016,259.0,4.0,AUTOMATIC,Sedan,27,19
2396,Volkswagen,CC,2015,280.0,6.0,AUTOMATED_MANUAL,Sedan,25,17
5494,Dodge,Grand Caravan,2016,283.0,6.0,AUTOMATIC,Passenger Minivan,25,17
...,...,...,...,...,...,...,...,...,...
5864,Subaru,Impreza,2015,148.0,4.0,MANUAL,4dr Hatchback,33,24
7924,Infiniti,QX50,2016,325.0,6.0,AUTOMATIC,4dr SUV,24,17
4719,Honda,Fit,2017,130.0,4.0,AUTOMATIC,4dr Hatchback,37,32
63,Audi,200,1991,162.0,5.0,AUTOMATIC,Sedan,20,16


In [12]:
from sklearn.feature_extraction import DictVectorizer

In [13]:
dv = DictVectorizer(sparse=False)

In [17]:
train_dicts = X_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [18]:
val_dicts = X_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
def rmse(y, y_pred):
  error = y-y_pred
  squared_error = error**2
  mean_squared_error =squared_error.mean()
  root_mean_squared_error=np.sqrt(mean_squared_error)
  return root_mean_squared_error

In [20]:
from sklearn.linear_model import Ridge


In [24]:
X_train.shape

(7148, 954)

In [28]:
len(set(dv.get_feature_names_out()))

954

In [31]:
for a in [0, 0.01, 0.1, 1, 10]:
  model= Ridge(solver='sag', random_state=42, alpha=a)
  model.fit(X_train,y_train)
  y_pred= model.predict(X_val)
  rmse_val= rmse(y_val,y_pred)
  print("rmse = %f , alpha=%f"%(rmse_val,a))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


rmse = 0.485990 , alpha=0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


rmse = 0.485991 , alpha=0.010000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


rmse = 0.485992 , alpha=0.100000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


rmse = 0.486010 , alpha=1.000000
rmse = 0.486191 , alpha=10.000000


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
